In [1]:
import pandas as pd
import numpy as np
import os
import requests
import json
import datetime
import time
import glob
import collections
import polars as pl 

In [2]:
COMP = 'lux-ai-season-3'
MAX_CALLS_PER_DAY = 100 # Kaggle says don't do more than 3600 per day and 1 per second
LOWEST_SCORE_THRESH = 1600

ROOT ="../working/"
META = "../data/meta/raw/"
MATCH_DIR = '../working/'
base_url = "https://www.kaggle.com/api/i/competitions.EpisodeService/"

get_url = base_url + "GetEpisodeReplay"
BUFFER = 1
COMPETITIONS = {
    'lux-ai-season-3' : 86411,
    'lux-ai-2022': 45040,
    'kore-2022': 34419,
    'lux-ai-2021': 30067,
    'hungry-geese': 25401,
    'rock-paper-scissors': 22838,
    'santa-2020': 24539,
    'halite': 18011,
    'google-football': 21723,
}

In [3]:
episodes_df = pl.read_csv(META + "Episodes.csv")
episodes_df = episodes_df.filter(pl.col('CompetitionId')==COMPETITIONS[COMP]) 
print(f'Episodes.csv: {len(episodes_df)} rows after filtering for {COMP}.')

Episodes.csv: 37839 rows after filtering for lux-ai-season-3.


In [11]:
epagents_df = pl.read_csv(META + "EpisodeAgents.csv", dtypes={'Reward':pl.Float32})
unique_comp_ids = pl.Series(episodes_df.select(pl.col("Id").unique())).to_list()
epagents_df = epagents_df.filter(pl.col("EpisodeId").is_in(unique_comp_ids))

print(f'EpisodeAgents.csv: {len(epagents_df)} rows after filtering for {COMP}.')

/var/folders/sn/rz8jym41349byvrhqn7h0rsr0000gn/T/ipykernel_33617/1327591740.py:1: DeprecationWarning: The argument `dtypes` for `read_csv` is deprecated. It has been renamed to `schema_overrides`.
  epagents_df = pl.read_csv(META + "EpisodeAgents.csv", dtypes={'Reward':pl.Float32})


EpisodeAgents.csv: 75650 rows after filtering for lux-ai-season-3.


In [13]:
episodes_df.head()

Id,Type,CompetitionId,CreateTime,EndTime
i64,i64,i64,str,str
58076100,4,86411,"""12/09/2024 19:50:26""","""12/09/2024 19:53:03"""
58078323,4,86411,"""12/09/2024 20:18:13""","""12/09/2024 20:20:45"""
58078642,4,86411,"""12/09/2024 20:22:44""","""12/09/2024 20:25:14"""
58079278,1,86411,"""12/09/2024 20:28:05""","""12/09/2024 20:30:37"""
58079282,4,86411,"""12/09/2024 20:31:23""","""12/09/2024 20:33:59"""


In [14]:
# preprocess
episodes_df = episodes_df.with_column(pl.col("CreateTime").cast(pl.Date64))

AttributeError: 'DataFrame' object has no attribute 'with_column'